In [1]:
import scipy.io as sio
import pandas as pd
import numpy as np
import glob
import csv
import json
import os

In [2]:
corpus_speech_list = ['BibleTTS/akuapem-twi',
    'BibleTTS/asante-twi',
    'BibleTTS/ewe',
    'BibleTTS/hausa',
    'BibleTTS/lingala',
    'BibleTTS/yoruba',
    'Buckeye',
    'EUROM',
    'LibriSpeech',
    # 'LibriVox',
    'MediaSpeech/AR',
    'MediaSpeech/ES',
    'MediaSpeech/FR',
    'MediaSpeech/TR',
    'MozillaCommonVoice/ab',
    'MozillaCommonVoice/ar',
    'MozillaCommonVoice/ba',
    'MozillaCommonVoice/be',
    'MozillaCommonVoice/bg',
    'MozillaCommonVoice/bn',
    'MozillaCommonVoice/br',
    'MozillaCommonVoice/ca',
    'MozillaCommonVoice/ckb',
    'MozillaCommonVoice/cnh',
    'MozillaCommonVoice/cs',
    'MozillaCommonVoice/cv',
    'MozillaCommonVoice/cy',
    'MozillaCommonVoice/da',
    'MozillaCommonVoice/de',
    'MozillaCommonVoice/dv',
    'MozillaCommonVoice/el',
    'MozillaCommonVoice/en',
    'MozillaCommonVoice/eo',
    'MozillaCommonVoice/es',
    'MozillaCommonVoice/et',
    'MozillaCommonVoice/eu',
    'MozillaCommonVoice/fa',
    'MozillaCommonVoice/fi',
    'MozillaCommonVoice/fr',
    'MozillaCommonVoice/fy-NL',
    'MozillaCommonVoice/ga-IE',
    'MozillaCommonVoice/gl',
    'MozillaCommonVoice/gn',
    'MozillaCommonVoice/hi',
    'MozillaCommonVoice/hu',
    'MozillaCommonVoice/hy-AM',
    'MozillaCommonVoice/id',
    'MozillaCommonVoice/ig',
    'MozillaCommonVoice/it',
    'MozillaCommonVoice/ja',
    'MozillaCommonVoice/ka',
    'MozillaCommonVoice/kab',
    'MozillaCommonVoice/kk',
    'MozillaCommonVoice/kmr',
    'MozillaCommonVoice/ky',
    'MozillaCommonVoice/lg',
    'MozillaCommonVoice/lt',
    'MozillaCommonVoice/ltg',
    'MozillaCommonVoice/lv',
    'MozillaCommonVoice/mhr',
    'MozillaCommonVoice/ml',
    'MozillaCommonVoice/mn',
    'MozillaCommonVoice/mt',
    'MozillaCommonVoice/nan-tw',
    'MozillaCommonVoice/nl',
    'MozillaCommonVoice/oc',
    'MozillaCommonVoice/or',
    'MozillaCommonVoice/pl',
    'MozillaCommonVoice/pt',
    'MozillaCommonVoice/ro',
    'MozillaCommonVoice/ru',
    'MozillaCommonVoice/rw',
    'MozillaCommonVoice/sr',
    'MozillaCommonVoice/sv-SE',
    'MozillaCommonVoice/sw',
    'MozillaCommonVoice/ta',
    'MozillaCommonVoice/th',
    'MozillaCommonVoice/tr',
    'MozillaCommonVoice/tt',
    'MozillaCommonVoice/ug',
    'MozillaCommonVoice/uk',
    'MozillaCommonVoice/ur',
    'MozillaCommonVoice/uz',
    'MozillaCommonVoice/vi',
    'MozillaCommonVoice/yo',
    'MozillaCommonVoice/yue',
    'MozillaCommonVoice/zh-CN',
    'MozillaCommonVoice/zh-TW',
    'primewords_chinese',
    'room_reader',
    'SpeechClarity',
    'TAT-Vol2',
    'thchs30',
    'TIMIT',
    'TTS_Javanese',
    'zeroth_korean',]

corpus_music_list = ['IRMAS',
                     'Albouy2020Science',
                     'CD',
                     'GarlandEncyclopedia']

In [3]:
def make_meta_file(corpus, corpus_type):
    
    params_list = glob.glob('STM_output/Survey/'+corpus_type+'_params_'+corpus+'/*')
    df_list = []

    # load the data from the mat file
    for params_file in params_list:
        # print(params_file)
        data_dict = sio.loadmat(params_file)
        structure_dict = {field: data_dict['Params'][field][0] for field in data_dict['Params'].dtype.names}
        df = pd.DataFrame(structure_dict)
        df.drop(columns=['x_axis','y_axis'], inplace=True)
        df = df.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
        df = df.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
        df['mat_filename'] = params_file.replace('/Survey/','/MATs/').replace('_params_','_mat_wl4_').replace('_Params.mat', '_MS2024.mat')
        df_list.append(df)
    
    df_all = pd.concat(df_list, ignore_index=True)

    # add the speaker ID and gender info
    if 'MozillaCommonVoice' in corpus:
        valid_df = pd.read_csv('data/speechCorp/'+corpus+'/validated.tsv', sep='\t', quoting=csv.QUOTE_NONE)
        valid_df['path'] = valid_df['path'].str.replace('.mp3', '')
        valid_df.rename(columns={'client_id':'speaker/artist'}, inplace=True)
        df_all = df_all.merge(valid_df[['speaker/artist', 'path', 'gender']], how='left', left_on='filename', right_on='path').drop(columns=['path'])
    elif 'BibleTTS' in corpus:
        df_all['speaker/artist'] = 'BibleTTS_' + df_all['filename'].str[:3]
        df_all['gender'] = np.nan
    elif 'Buckeye' in corpus:
        df_all['speaker/artist'] = df_all['filename'].str[:3].str.replace('s', 'S')
        df_gender = pd.read_csv('data/speechCorp/Buckeye/Buckeye_speaker_info.csv')
        df_all = df_all.merge(df_gender[['SPEAKER', "SPEAKER'S GENDER"]], how='left', left_on='speaker/artist', right_on='SPEAKER').drop(columns=['SPEAKER'])
        df_all.rename(columns={"SPEAKER'S GENDER":'gender'}, inplace=True)
        df_all['speaker/artist'] = 'Buckeye_'+df_all['speaker/artist']
    elif 'EUROM' in corpus:
        df_all['speaker/artist'] = 'EUROM_' + df_all['LangOrInstru'] +'_'+ df_all['filename'].str[:2]
        df_all['gender'] = np.nan
    elif 'MediaSpeech' in corpus:
        df_all['speaker/artist'] = 'MediaSpeech_' + df_all['LangOrInstru'] +'_'+ df_all['filename']
        df_all['gender'] = np.nan
    elif 'LibriSpeech' in corpus:
        split_names = df_all['filename'].str.split('-') # Split the 'name' column by "-"
        first_parts = split_names.str[0] # Extract the first part of the split result
        df_all['speaker/artist'] = first_parts # Add the extracted part as a new column in the DataFrame
        
        ## load LibriSpeech text file
        reader_ids = []
        genders = []
        subsets = []
        durations = []
        names = []
        # Open the file and read line by line
        with open('data/speechCorp/LibriSpeech/SPEAKERS.TXT', 'r') as file:
            # Skip lines starting with ";" (comments) until reaching the data
            while True:
                line = file.readline()
                if not line.startswith(';'):
                    # Start processing data from this line
                    break
            
            # Read the rest of the lines and parse them
            while line:
                # Split each line by "|" character
                data = line.strip().split('|')
                # Extract relevant information
                reader_ids.append(int(data[0].strip()))
                genders.append(data[1].strip())
                subsets.append(data[2].strip())
                durations.append(float(data[3].strip()))
                names.append(data[4].strip())
                # Read the next line
                line = file.readline()
        
        # Create a DataFrame using the lists
        df_LibriSpeech = pd.DataFrame({'reader_id': reader_ids, 'gender': genders, 'subset': subsets, 'duration': durations, 'name': names})
        df_LibriSpeech['reader_id']= df_LibriSpeech['reader_id'].astype(str)
        df_all = df_all.merge(df_LibriSpeech[['reader_id', 'gender']], how='left', left_on='speaker/artist', right_on='reader_id').drop(columns=['reader_id'])
    elif 'primewords_chinese' in corpus:
        with open('data/speechCorp/primewords_chinese/set1_transcript.json', 'r') as file:
            data = json.load(file)
        primewords_df = pd.DataFrame(data)
        primewords_df['file'] = primewords_df['file'].str.replace('.wav', '')
        df_all = df_all.merge(primewords_df[['file', 'user_id']], how='left', left_on='filename', right_on='file').drop(columns=['file'])
        df_all.rename(columns={"user_id":'speaker/artist'}, inplace=True)
        df_all['speaker/artist'] = 'primewords_'+df_all['speaker/artist']
        df_all['gender']=np.nan
    elif 'room_reader' in corpus:
        split_names = df_all['filename'].str.split('_') # Split the 'name' column by "-"
        df_all['speaker/artist'] = split_names.str[1] # Extract the first part of the split result
        RR_df = pd.read_excel('data/speechCorp/room_reader/RoomReader_SessionsEvents.xlsx')
        df_all = df_all.merge(RR_df[['part_ID', 'gender']], how='left', left_on='speaker/artist', right_on='part_ID').drop(columns=['part_ID'])
        df_all['speaker/artist'] = 'RoomReader_'+df_all['speaker/artist']
    elif 'SpeechClarity' in corpus:
        df_all['speaker/artist'] = 'SpeechClarity_'+df_all['filename'].str[:3]
        df_all['gender'] = np.nan
    elif 'TAT-Vol2' in corpus:
        df_all['speaker/artist'] = 'TAT-Vol2_'+df_all['filename'].str[:10]
        df_all['gender'] = df_all['filename'].str[5]
    elif 'thchs30' in corpus:
        split_names = df_all['filename'].str.split('_') # Split the 'name' column by "-"
        df_all['speaker/artist'] = 'thchs30_'+split_names.str[0] # Extract the first part of the split result
        df_all['gender'] = np.nan
    elif 'TIMIT' in corpus:
        split_names = df_all['filepath'].str.split('/') # Split the 'name' column by "-"
        df_all['speaker/artist'] = 'TIMIT_'+split_names.str[-2] # Extract the first part of the split result
        df_all['gender'] = df_all['speaker/artist'].str[6]
    elif 'TTS_Javanese' in corpus:
        df_all['speaker/artist'] = df_all['filename'].str[:9] 
        df_all['gender'] = df_all['filename'].str[2]
    elif 'zeroth' in corpus:
        df_all['speaker/artist'] = df_all['filename'].str[:3] 
        zeroth_df = pd.read_csv('data/speechCorp/zeroth_korean/AUDIO_INFO', sep="|")
        zeroth_df['SPEAKERID'] = zeroth_df['SPEAKERID'].astype(str)
        df_all = df_all.merge(zeroth_df[['SPEAKERID', 'SEX']], how='left', left_on='speaker/artist', right_on='SPEAKERID').drop(columns=['SPEAKERID'])
        df_all.rename(columns={"SEX":'gender'}, inplace=True)
        df_all['speaker/artist'] = 'zeroth_'+df_all['speaker/artist']

    # Music corpora
    elif 'IRMAS' in corpus:
        for nRow in range(len(df_all)):
            txt_path = df_all['filepath'][nRow].replace('.wav', '.txt')
            if os.path.exists(txt_path):
                # Open the file in read mode
                with open(txt_path, 'r') as file:
                    # Read all lines into a list
                    lines = file.readlines()
                converted_lines = [line.strip() for line in lines]
                df_all.loc[nRow,'LangOrInstru'] = '-'.join(converted_lines)
                df_all.loc[nRow,'VoiOrNot'] = int('voi' in converted_lines)
                
        # df_all['speaker/artist'] = '-'.join(df_all['filename'].split('-')[:-1]).strip()
        df_all['speaker/artist'] = df_all['filename'].apply(lambda x: '-'.join(x.split('-')[:-1]).strip())
        df_all['gender'] = np.nan
        df_all['genre'] = np.nan
        
    elif 'Albouy2020Science' in corpus:
        df_all['speaker/artist'] = 'Albouy2020Science'
        df_all['gender'] = 'female'
        df_all['genre'] = 'classical'
        df_all['LangOrInstru'] = 'English'
        df_all.loc[df_all['filename'].str.contains('French', case=False), 'LangOrInstru'] = 'French'
        df_all['VoiOrNot'] = 1

    elif 'CD' in corpus:
        from fuzzywuzzy import process
        df_all['gender'] = np.nan
        def extract_artist(file_path):
            parts = file_path.split('/')
            artist_album_part = parts[3]
            return artist_album_part.split('_')[0]
        
        df_all['speaker/artist'] = df_all['filepath'].apply(extract_artist)
        df_CD = pd.read_excel('data/musicCorp/CD/CD_music_list.xlsx')

        # Function to find the best match for each name in df1 from df2
        def find_best_match(name, choices):
            return process.extractOne(name, choices)
        
        df_all['Best_Match'] = df_all['filename'].apply(lambda x: find_best_match(x, df_CD['Piece'])) # Apply the function to find the best match for each name in df1
        df_all['Matched_Name'] = df_all['Best_Match'].apply(lambda x: x[0]) # Extract matched names and similarity scores
        df_all['Similarity_Score'] = df_all['Best_Match'].apply(lambda x: x[1])
        
        # Join based on matched names
        df_all = pd.merge(df_all, df_CD[['Piece', 'Genre','Instrument']], left_on='Matched_Name', right_on='Piece', how='left')
        df_all['LangOrInstru'] = df_all['Instrument']
        df_all.drop(columns=['Best_Match','Instrument','Matched_Name','Similarity_Score','Piece'], inplace = True)
        df.rename(columns={'Genre': 'genre'}, inplace = True)

        df_all['VoiOrNot'] = 0
        df_all.loc[df_all['LangOrInstru'].str.contains('Voi', case=False), 'VoiOrNot'] = 1

        df_all = df_all[~df_all['filepath'].str.contains('Compilations', case=False)]
        
    elif 'GarlandEncyclopedia' in corpus:
        df_all['speaker/artist'] = df_all['filename']
        df_all['gender'] = np.nan
        df_all['LangOrInstru'] = np.nan
        df_all['genre'] = 'world'
        Garland_novoice_list = list(pd.read_csv('data/musicCorp/GarlandEncyclopedia/Garland_noVoice.csv', header=None)[0])
        df_all['VoiOrNot'] = 1
        df_all.loc[df_all['filename'].isin(Garland_novoice_list), 'VoiOrNot'] = 0

    return df_all

In [4]:
for corpus in corpus_speech_list:
    print(corpus)
    df_all = make_meta_file(corpus, corpus_type='speech')
    df_all.to_csv('STM_output/STM_metaData/metaData_'+corpus.replace('/', '-')+'.csv')


BibleTTS/akuapem-twi
BibleTTS/asante-twi
BibleTTS/ewe
BibleTTS/hausa
BibleTTS/lingala
BibleTTS/yoruba
Buckeye
EUROM
LibriSpeech
MediaSpeech/AR
MediaSpeech/ES
MediaSpeech/FR
MediaSpeech/TR
MozillaCommonVoice/ab
MozillaCommonVoice/ar


/var/folders/fw/slnm06yn15lgwlcg_7jpt3t00000gn/T/ipykernel_7265/3234401510.py:22: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  valid_df = pd.read_csv('data/speechCorp/'+corpus+'/validated.tsv', sep='\t', quoting=csv.QUOTE_NONE)


MozillaCommonVoice/ba


/var/folders/fw/slnm06yn15lgwlcg_7jpt3t00000gn/T/ipykernel_7265/3234401510.py:22: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  valid_df = pd.read_csv('data/speechCorp/'+corpus+'/validated.tsv', sep='\t', quoting=csv.QUOTE_NONE)


MozillaCommonVoice/be


...

MozillaCommonVoice/ur
MozillaCommonVoice/uz


/var/folders/fw/slnm06yn15lgwlcg_7jpt3t00000gn/T/ipykernel_7265/3234401510.py:22: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  valid_df = pd.read_csv('data/speechCorp/'+corpus+'/validated.tsv', sep='\t', quoting=csv.QUOTE_NONE)


MozillaCommonVoice/vi
MozillaCommonVoice/yo
MozillaCommonVoice/yue
MozillaCommonVoice/zh-CN
MozillaCommonVoice/zh-TW


/var/folders/fw/slnm06yn15lgwlcg_7jpt3t00000gn/T/ipykernel_7265/3234401510.py:22: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  valid_df = pd.read_csv('data/speechCorp/'+corpus+'/validated.tsv', sep='\t', quoting=csv.QUOTE_NONE)


primewords_chinese
room_reader
SpeechClarity
TAT-Vol2
thchs30
TIMIT
TTS_Javanese
zeroth_korean


In [4]:
for corpus in corpus_music_list:
    print(corpus)
    df_all = make_meta_file(corpus, corpus_type='music')
    df_all.to_csv('STM_output/STM_metaData/metaData_'+corpus.replace('/', '-')+'.csv')

IRMAS
Albouy2020Science
CD
GarlandEncyclopedia


## test code ##

In [24]:
corpus = 'GarlandEncyclopedia'

corpus_type='music'

params_list = glob.glob('STM_output/Survey/'+corpus_type+'_params_'+corpus+'/*')
df_list = []

# load the data from the mat file
for params_file in params_list:
    # print(params_file)
    data_dict = sio.loadmat(params_file)
    structure_dict = {field: data_dict['Params'][field][0] for field in data_dict['Params'].dtype.names}
    df = pd.DataFrame(structure_dict)
    df.drop(columns=['x_axis','y_axis'], inplace=True)
    df = df.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
    df = df.applymap(lambda x: x[0] if isinstance(x, (list, np.ndarray)) else x)
    df['mat_filename'] = params_file.replace('/Survey/','/MATs/').replace('_params_','_mat_wl4_').replace('_Params.mat', '_MS2024.mat')
    df_list.append(df)

df_all = pd.concat(df_list, ignore_index=True)
df_all

,filepath,filename,LangOrInstru,VoiOrNot,startPoint,endPoint,FSkhz,expdur,maxsil,totalLengCur,mat_filename
0,data/musicCorp/GarlandEncyclopedia/52 Marquesa...,52 Marquesan Pig Dance (Maha_u),nonIRMAS,NaN,1,2116800,44.1,48,0.482109,48,STM_output/MATs/music_mat_wl4_GarlandEncyclope...
1,data/musicCorp/GarlandEncyclopedia/02 Ethiopia...,02 Ethiopian Lidet Celebration,nonIRMAS,NaN,1,5292000,44.1,120,0.780907,120,STM_output/MATs/music_mat_wl4_GarlandEncyclope...
2,data/musicCorp/GarlandEncyclopedia/28 A La Cla...,28 A La Claire Fontaine,nonIRMAS,NaN,1,4233600,44.1,96,0.378798,92,STM_output/MATs/music_mat_wl4_GarlandEncyclope...
3,data/musicCorp/GarlandEncyclopedia/8-07 Wales_...,8-07 Wales_ Penillion,nonIRMAS,NaN,1,3880800,44.1,88,0.877823,88,STM_output/MATs/music_mat_wl4_GarlandEncyclope...
4,data/musicCorp/GarlandEncyclopedia/20 Yemeni V...,20 Yemeni Vocal Music With Lute Accompaniment,nonIRMAS,NaN,1,5292000,44.1,120,0.365306,120,STM_output/MATs/music_mat_wl4_GarlandEncyclope...
...,...,...,...,...,...,...,...,...,...,...,...
291,data/musicCorp/GarlandEncyclopedia/39 Nissan M...,39 Nissan Music For Masked Dancer,nonIRMAS,NaN,1,3528000,44.1,80,0.744671,76,STM_output/MATs/music_mat_wl4_GarlandEncyclope...
292,data/musicCorp/GarlandEncyclopedia/16 _Boa Noi...,16 _Boa Noite Todo Povo_ (_Good Night Everyone...,nonIRMAS,NaN,1,4762800,44.1,108,0.255329,104,STM_output/MATs/music_mat_wl4_GarlandEncyclope...
293,data/musicCorp/GarlandEncyclopedia/08 _Walata ...,08 _Walata Grande_,nonIRMAS,NaN,1,3528000,44.1,80,0.298639,80,STM_output/MATs/music_mat_wl4_GarlandEncyclope...
294,"data/musicCorp/GarlandEncyclopedia/16 _Anawa, ...","16 _Anawa, Anawa_ Kiribati Women_s Hip-Shaking...",nonIRMAS,NaN,1,3880800,44.1,88,0.722993,84,STM_output/MATs/music_mat_wl4_GarlandEncyclope...


In [29]:
import os
import scipy.io as sio
import pandas as pd
import numpy as np
import glob
import csv
import json


        
# df_all['speaker/artist'] = '-'.join(df_all['filename'].split('-')[:-1]).strip()
df_all['speaker/artist'] = df_all['filename']
df_all['gender'] = np.nan
df_all['LangOrInstru'] = np.nan
df_all['genre'] = 'world'
df_all.head()

,filepath,filename,LangOrInstru,VoiOrNot,startPoint,endPoint,FSkhz,expdur,maxsil,totalLengCur,mat_filename,speaker/artist,gender,genre
0,data/musicCorp/GarlandEncyclopedia/52 Marquesa...,52 Marquesan Pig Dance (Maha_u),NaN,1,1,2116800,44.1,48,0.482109,48,STM_output/MATs/music_mat_wl4_GarlandEncyclope...,52 Marquesan Pig Dance (Maha_u),NaN,world
1,data/musicCorp/GarlandEncyclopedia/02 Ethiopia...,02 Ethiopian Lidet Celebration,NaN,1,1,5292000,44.1,120,0.780907,120,STM_output/MATs/music_mat_wl4_GarlandEncyclope...,02 Ethiopian Lidet Celebration,NaN,world
2,data/musicCorp/GarlandEncyclopedia/28 A La Cla...,28 A La Claire Fontaine,NaN,1,1,4233600,44.1,96,0.378798,92,STM_output/MATs/music_mat_wl4_GarlandEncyclope...,28 A La Claire Fontaine,NaN,world
3,data/musicCorp/GarlandEncyclopedia/8-07 Wales_...,8-07 Wales_ Penillion,NaN,1,1,3880800,44.1,88,0.877823,88,STM_output/MATs/music_mat_wl4_GarlandEncyclope...,8-07 Wales_ Penillion,NaN,world
4,data/musicCorp/GarlandEncyclopedia/20 Yemeni V...,20 Yemeni Vocal Music With Lute Accompaniment,NaN,1,1,5292000,44.1,120,0.365306,120,STM_output/MATs/music_mat_wl4_GarlandEncyclope...,20 Yemeni Vocal Music With Lute Accompaniment,NaN,world


In [30]:
Garland_novoice_list = list(pd.read_csv('data/musicCorp/GarlandEncyclopedia/Garland_noVoice.csv', header=None)[0])
Garland_novoice_list

['01 Eastern Arab Art Music',
 '01 The Green Lotus',
 '01 The Sound Of Frogs (Thailand)',
 '02 Chinese Ensemble With Song _Chung We Meng_ (Thailand)',
 '02 Jor Improv On Vicitra Vina In Rag Bairagi',
 '02 Tannese Conch-Shell Trio',
 '02 Yangguan Sandie',
 '03 Kao Shan Liu Shui [Gao Shan Liushui] _High Mountain, Flowing Stream_',
 '03 Kyal Ang Gat',
 '03 Pohnpei Kava-Poinding Rhythms (Sukusuk)',
 '03 Turkish Classical',
 '04 Classical Persian',
 '04 Moonlight On The Ching Yang River',
 '04 Salaw Seung Pi Ensemble (Northern Thailand)',
 '05 Raga Hamsanandi',
 '05 Sikuriada (Music For Siku Panpipes)',
 '06 _Kulwa_ (Kupla, Cobla)',
 '06 Yemeni Highland Drumming',
 '06 Yu Wo Lang _The Fish Lying On The Wave__ Xiao Fangniu _The Small Shepherd_',
 '07 Hsiu Ho Pao [Xiu Hebao] _An Embroidered Purse_',
 '07 Lichiwayu Notch Flutes',
 '07 Yemeni Drumming From The Hadramawt',
 '08 _Walata Grande_',
 '08 Yemeni Drumming From The Tihama',
 '8-01 Jewish Instrumental Melodies',
 '8-06 England_ Northumb

In [27]:
df_all['VoiOrNot'] = 1
df_all.loc[df_all['filename'].isin(Garland_novoice_list), 'VoiOrNot'] = 0

In [28]:
df_all

,filepath,filename,LangOrInstru,VoiOrNot,startPoint,endPoint,FSkhz,expdur,maxsil,totalLengCur,mat_filename,speaker/artist,gender,genre
0,data/musicCorp/GarlandEncyclopedia/52 Marquesa...,52 Marquesan Pig Dance (Maha_u),NaN,1,1,2116800,44.1,48,0.482109,48,STM_output/MATs/music_mat_wl4_GarlandEncyclope...,52 Marquesan Pig Dance (Maha_u),NaN,world
1,data/musicCorp/GarlandEncyclopedia/02 Ethiopia...,02 Ethiopian Lidet Celebration,NaN,1,1,5292000,44.1,120,0.780907,120,STM_output/MATs/music_mat_wl4_GarlandEncyclope...,02 Ethiopian Lidet Celebration,NaN,world
2,data/musicCorp/GarlandEncyclopedia/28 A La Cla...,28 A La Claire Fontaine,NaN,1,1,4233600,44.1,96,0.378798,92,STM_output/MATs/music_mat_wl4_GarlandEncyclope...,28 A La Claire Fontaine,NaN,world
3,data/musicCorp/GarlandEncyclopedia/8-07 Wales_...,8-07 Wales_ Penillion,NaN,1,1,3880800,44.1,88,0.877823,88,STM_output/MATs/music_mat_wl4_GarlandEncyclope...,8-07 Wales_ Penillion,NaN,world
4,data/musicCorp/GarlandEncyclopedia/20 Yemeni V...,20 Yemeni Vocal Music With Lute Accompaniment,NaN,1,1,5292000,44.1,120,0.365306,120,STM_output/MATs/music_mat_wl4_GarlandEncyclope...,20 Yemeni Vocal Music With Lute Accompaniment,NaN,world
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,data/musicCorp/GarlandEncyclopedia/39 Nissan M...,39 Nissan Music For Masked Dancer,NaN,1,1,3528000,44.1,80,0.744671,76,STM_output/MATs/music_mat_wl4_GarlandEncyclope...,39 Nissan Music For Masked Dancer,NaN,world
292,data/musicCorp/GarlandEncyclopedia/16 _Boa Noi...,16 _Boa Noite Todo Povo_ (_Good Night Everyone...,NaN,1,1,4762800,44.1,108,0.255329,104,STM_output/MATs/music_mat_wl4_GarlandEncyclope...,16 _Boa Noite Todo Povo_ (_Good Night Everyone...,NaN,world
293,data/musicCorp/GarlandEncyclopedia/08 _Walata ...,08 _Walata Grande_,NaN,0,1,3528000,44.1,80,0.298639,80,STM_output/MATs/music_mat_wl4_GarlandEncyclope...,08 _Walata Grande_,NaN,world
294,"data/musicCorp/GarlandEncyclopedia/16 _Anawa, ...","16 _Anawa, Anawa_ Kiribati Women_s Hip-Shaking...",NaN,1,1,3880800,44.1,88,0.722993,84,STM_output/MATs/music_mat_wl4_GarlandEncyclope...,"16 _Anawa, Anawa_ Kiribati Women_s Hip-Shaking...",NaN,world
